## Cholesky QR decomposition of a Tall and Skinny matrix

N.B. This is not the direct TSQR method proposed in the article. The Cholesky-based approach lacks precision and numerical stability, making it impractical for high-performance environments. Nevertheless, we found it interesting to implement it as an exercise, exploring the Dask implementation and benchmarking it, despite its inherent numerical instability

Let $B$ be a symmetric and positive definite $n\times n$ matrix. Then its Cholesky decomposition is:
$$
B = L L^T 
$$
where $L$ is a lower triangular $n\times n$ matrix. Cholesky decomposition turns out to be particularly useful when computing the QR decomposition of a $m\times n$ matrix $A$. Let's first build the temporary matrix $T = A^T A$, symmetric by definition. Hence, if:
$$
A = QR
$$
then:
$$
T = A^T A = (QR)^T(QR) = R^T Q^T Q R
$$
The matrix $Q$ is orthogonal, so $T = R^T R$. Since $R$ is a triangular matrix too, then we have effectively found the Cholesky decomposition of $T$ in terms of $R$. In other words, the Cholesky QR decomposition proceeds as follow:
1) Given $A$, build the symmetric matrix $T = A^T A$
2) Apply the Cholesky decomposition on $ T = L L^T$
3) Obtain $R = L^T$. Obtain Q by solving $A = QR$

This procedure is numerically unstable, and for certain matrices $A$ it may fail to produce accurate results, particularly for the orthogonal matrix $Q$. However, it is easily parallelizable, making it a useful testbed for experimenting with Dask.


In [1]:
# CLUSTER DEPLOYMENT, TO BE EXECUTED ONLY IN A LOCAL ENVIRONMENT!!
from dask.distributed import Client, LocalCluster

# For now, local deployment on my computer (multicore)
ncore = 4
cluster = LocalCluster(n_workers=ncore, threads_per_worker=1)
client = Client(cluster)

# Print the dashboard link over the port 8787
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [ ]:
# CLUSTER DEPLOYMENT ON CLOUDVENETO
from dask.distributed import Client, SSHCluster

cluster = SSHCluster(
    ["10.67.22.154", "10.67.22.216", "10.67.22.116", "10.67.22.113"],
    connect_options={"known_hosts": None},
    remote_python="/home/ubuntu/miniconda3/bin/python",
    scheduler_options={"port": 8786, "dashboard_address": ":8797"},
    worker_options={
        "nprocs": 1,     
        "nthreads": 1  
    }
)

client = Client(cluster)

In [2]:
# check if everything went smoothly
print(client)

<Client: 'tcp://127.0.0.1:42685' processes=4 threads=4, memory=13.49 GiB>


Import the necessary stuff along with the California housing dataset

In [ ]:
import dask.array as da
import dask
import numpy as np
from sklearn.datasets import fetch_california_housing

# Download California Housing dataset
data = fetch_california_housing(as_frame=True)

# Convert features into Dask Array (it's a matrix).
n_partition = 3        # number of partition in memory. We have 4 VMS (1 master + 3 workers), so let's start with just 3 partitions
length_partition = data.data.shape[0] // n_partition
X_da = da.from_array(data.data.values, chunks=(length_partition, data.data.shape[1]))

print("Number of Dask partitions:",  X_da.npartitions) 
print("Length of each partition:", length_partition, "rows")
print("Length of the whole dataset:", data.data.shape[0], "rows")

Number of Dask partitions: 4
Length of each partition: 5160 rows
Length of the whole dataset: 20640 rows


Now we'll define the parallel and serial algorithm for the Cholesky QR decomposition.

This first parallel version of the Cholesky method works as follows:
1) The array should already be splitted by rows in partitions across workers (let's call each partition $A_p$). Each worker computes a local version of $A^T A$, i.e. $A_p^T A_p$. Since $A_p$ is smaller than $A$, the matrix multiplication should proceed faster. Furthermore, $A_p$ being smaller may fully reside in the RAM of a worker
2) Once each worker has finished, the full Gram matrix $A^T A$ is computed in a single worker by summing up all the smaller and local $A_p$: $A^T A = \sum_p A_P^T A_p$
3) The matrix $A^T A$ is small, $n\times n$. A serial Cholesky decomposition is performed and will output the final $R$ matrix
4) To get $Q$, we will use the defining equation $A = QR \Rightarrow Q = A R^{-1}$. Computing the inverse of $R$ is straightforward and can be done by a single worker, whereas the MatMul between $A$ and the inverse of $R$ can be parallelized

In [7]:
def compute_choleskyQR_parallel(X_da : dask.array.Array):
    # A list of delayed tasks for each partition of the dataset
    # Each partition computes the local Gram matrix (as a delayed task)
    chunks_delayed = [dask.delayed(lambda x : x.T @ x)(chunk) for chunk in X_da.to_delayed().ravel()]

    # Now sum all the local Gram matrices to get the global Gram matrix
    Gram_global_delayed = dask.delayed(sum)(chunks_delayed)   ## !! This is not strictly parallel, meaning that a single worker will perform the sum instead of a tree-like operation. This is ok here, I guess, since we only have 8 chunks that need to be summed up

    # Compute R as the Cholesky decomposition on the global Gram matrix (as a delayed even if a serial operation just call .compute at the end)
    R = dask.delayed(np.linalg.cholesky)(Gram_global_delayed)
    #R.visualize("fig/CholeskyR.png")
    R = R.compute() # Compute R. This will put a stop at the parallel operation
    R_inv = np.linalg.inv(R) # It's a small matrix, so this operation is fast even if serial

    Q = X_da.map_blocks(lambda block: block @ R_inv, dtype=X_da.dtype)
    #Q.visualize("fig/CholeskyQ.png")
    Q = Q.compute() # Compute Q
    return Q, R

def compute_choleskyQR_serial(X):
    # Global gram matrix
    G = X.T @ X
    R = np.linalg.cholesky(G)
    R_inv = np.linalg.inv(R)
    Q = X @ R_inv
    
    return Q, R

def compute_choleskyR_parallel(X_da : dask.array.Array):
    # A list of delayed tasks for each partition of the dataset
    # Each partition computes the local Gram matrix
    chunks_delayed = [dask.delayed(lambda x : x.T @ x)(chunk) for chunk in X_da.to_delayed().ravel()]
    # Now sum all the local Gram matrices to get the global Gram matrix
    Gram_global_delayed = dask.delayed(sum)(chunks_delayed)
    # Compute R as the Cholesky decomposition on the global Gram matrix (as a delayed even if a serial operation just call .compute at the end)
    R = dask.delayed(np.linalg.cholesky)(Gram_global_delayed)
    R = R.compute() # Compute R
    return  R

def compute_choleskyR_serial(X):
    # Global gram matrix
    G = X.T @ X
    R = np.linalg.cholesky(G)
    return R

The DAG should look like (for the computation of R)


![](fig/CholeskyR.png)

Let's measure the time it takes to perform the parallel Cholesky QR decomposition:

In [33]:
%%time
# parallel
Q_p, R_p = compute_choleskyQR_parallel(X_da)

CPU times: user 44.1 ms, sys: 8.66 ms, total: 52.8 ms
Wall time: 65 ms


As of now, we have 3 VMs and we specifically asked Dask to only create one worker per node, thus we have deployed 3 workers. Accessing the dashboard, we can see what happens under the hood:

![](fig/CloudVeneto_Cal_3workers.png)

Since we have 3 workers, we will see three horizontal stripes corresponding to each worker. The first three bands (green-ish) are labeled *"array"* by Dask and are probably related to array accessing and reading. The following parallel blocks (three of them, as expected) represents the *lambda* function, i.e. the local matrix multiplication. The red block (and the following yellowe one) is performed on a single worker as requested. They represent the serial sum: a single worker collects all the temporary Gram matrices (red block) and perform the sum operation (yellow). All the gaps in between the colored bands represent the Dask overhead (orchestration, ...) which, in this specific case, looks like a lot of time. In fact, running the same algorithm in a serial fashion: 

Workers were idle most of the time. Additionally, there are some long transfers (red)

In [35]:
%%time
# serial
Q_s, R_s = compute_choleskyQR_serial(data.data.values)

CPU times: user 822 µs, sys: 721 µs, total: 1.54 ms
Wall time: 847 µs


However, the serial algorithm is faster than the parallel one. This is probably because the California dataset is not that big (only $20k$ rows, easily fittable in the RAM). As of now, we still can't see a real speedup

Cholesky QR is sadly known to be unstable. In fact:

In [ ]:
# Let's see whether the results are compatible
diffR = np.linalg.norm(R_p - R_s, 2)
diffQ = np.linalg.norm(Q_p - Q_s, 2)
print(f"||R_parallel - R_serial||_2 = {diffR}")
print(f"||Q_parallel - Q_serial||_2 = {diffQ}")

# Check orthogonality of Q
orthogonality_metric = np.linalg.norm(Q_s.T @ Q_s - np.eye(Q_s.shape[1]), 2)
print(f"||Q^T @ Q- I||_2 = {orthogonality_metric}")
# Check decomposition
decomp_metric = np.linalg.norm(data.data.values - Q_s @ R_s, 2)
print(f"||X - Q @ R||_2 = {decomp_metric}")

As expected, the decomposition yielded a non reasonnable result (Q is not orthogonal, the algorithm is highly unstable)


Let's try with a different and larger dataset (HIGGS dataset)

In [ ]:
import dask.dataframe as dd

# A huge dataset
df = dd.read_csv("HIGGS.csv", header=None, blocksize="400MB")
X_df = df.iloc[:, 1:] 
X_da = X_df.to_dask_array(lengths=True)

In [ ]:
%%time
Q, R = compute_choleskyQR_parallel(X_da)

In [34]:
client.close()